Librerie varie da installare

In [ ]:
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn

Inclusione delle librerie utilizzate

In [ ]:
import pandas as pd
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Variabili di gestione files e selezione colonne dataset

In [ ]:
# Path of the file, can be changed
PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset\Attive_rielaborate_G"

# True = export summary file in the OUTPUT_PATH
to_export = False

# Path of the desired output file, can be changed
OUTPUT_PATH = r"C:\Users\Andre\OneDrive - Università degli Studi di Parma\Tirocinio\Dataset_filtrato"

# Columns to be kept, discard all the other ones
to_keep_columns = ['Ragione sociale', 'Province', 'Number of employees', 'Accounting closing date',
                   'Legal Form', 'Legal Status ', 'TOTAL receivables due from shareholders',
                   'Called share capital', 'TOTAL FIXED ASSETS\n', 'TOTAL INTANGIBLE FIXED ASSETS',
                   'TOTAL TANGIBLE FIXED ASSETS', 'TOTAL FINANCIAL FIXED ASSETS', 'TOTAL CURRENT ASSETS',
                   'TOTAL FINANCIAL ASSETS', 'TOTAL ACCRUED INCOME AND PREPAID EXPENSES', 'Total asset',
                   "TOTAL SHAREHOLDERS' FUNDS", 'TOTAL PROVISIONS FOR RISKS AND CHARGES', 'SEVERANCE INDEMNITY RESERVE',
                   'TOTAL PAYABLES', 'TOTAL VALUE OF PRODUCTION', 'TOTAL PRODUCTION COSTS' ,'OPERATING MARGIN',
                   'Added Value', 'TOTAL FINANCIAL INCOME AND CHARGE', 'TOTAL FINANCIAL ASSETS ADJUSTMENTS',
                   'PROFIT/LOSS BEFORE TAXATION', 'Employees']

Leggo, per esempio, i file .csv contenuti in una directory e le sue sottodirectories

In [ ]:
# Dataset with all the csv files
input_dataset = pd.DataFrame()

# Path of each csv file
# Path descrption: Attive_rielaborate_G/2015|2016|2017/*.csv
csv_files = glob.glob(os.path.join(PATH, "**/*.csv"), recursive=True)

# Loop over the list of csv files
for f in csv_files:

    # Print the location and filename
    print('Location:', f)
    print('File Name:', f.split("\\")[-1])

    # Read each csv file
    partial_dataset = pd.read_csv(f, sep=";", dtype=object, thousands=",", decimal=".", usecols=to_keep_columns)

    # Append the partial dataset
    input_dataset = pd.concat([input_dataset, partial_dataset])




print("Dimensione dataset: ", input_dataset.shape)

Rinomino colonne con caratteri speciali o typo

In [ ]:
columns_map = {"TOTAL FIXED ASSETS\n": "TOTAL FIXED ASSETS",
              "Legal Status ": "Legal Status",
              "TOTAL SHAREHOLDERS' FUNDS": "TOTAL SHAREHOLDERS FUNDS"}
input_dataset.rename(columns=columns_map, inplace=True)

Cerco valori mancanti

In [ ]:
input_dataset.replace(to_replace="n.a.", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="n.d.", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="#VALUE!", value=np.NaN, inplace=True)
input_dataset.replace(to_replace="UDATACHI", value=np.NaN, inplace=True)
input_dataset.isna().sum()

Filtro i valori non disponibili

In [ ]:
input_dataset = input_dataset.dropna()
input_dataset.isna().sum()

In [ ]:
print("Dimensione dataset filtrato dai valori mancanti e con numer odi colonne ridotte: ", input_dataset.shape)
input_dataset.head()

Cast delle colonne a tipi di dato sensati

In [ ]:
str_cols = ['Ragione sociale', 'Province',
            'Legal Form', 'Legal Status',
            'Accounting closing date']


int_cols = ['Number of employees', 'Employees']



for col in input_dataset.columns:
    if col in str_cols:
        input_dataset[col] = input_dataset[col].astype(str)
    else:
        input_dataset[col] = input_dataset[col].str.replace(',','')
        input_dataset[col] = pd.to_numeric(input_dataset[col])

Sposto le colonne non numeriche

In [ ]:
complete_dataset = input_dataset.copy()
descriptive_dataset = input_dataset[str_cols]
input_dataset.drop(str_cols, axis=1, inplace=True)

Guardo le statistiche delle restanti colonne

In [ ]:
pd.set_option('display.float_format', lambda x: '%i' % x)
input_dataset.describe(include="all").T

In [ ]:
descriptive_dataset.describe().T

Piccola panoramica del dataset filtrato

In [ ]:
complete_dataset

Esporto il file riepilogativo in formato xlsx e csv, se richiesto

In [ ]:
# Around 10 minutes for the xlsx file
if to_export:
    #complete_dataset.to_excel(OUTPUT_PATH + "summary.xlsx")
    complete_dataset.to_csv(OUTPUT_PATH + "/summary.csv")

Qualche analisi sulla correlazione dei dati

In [ ]:
sns.set_theme(style="white")

# Compute the correlation matrix
corr = input_dataset.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Rimuovo colonne fortemente correlate

In [ ]:
correlated_cols = ['TOTAL SHAREHOLDERS FUNDS', 'Added Value',
                   'Employees', 'Total asset',
                   'TOTAL PRODUCTION COSTS', 'TOTAL PAYABLES',
                   'TOTAL FINANCIAL FIXED ASSETS']
input_dataset.drop(columns= correlated_cols, axis=1, inplace=True)

In [ ]:
# Compute the correlation matrix
corr = input_dataset.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1,
            center=0, annot=True, fmt='.2f',
            square=True, linewidths=.5, cbar_kws={"shrink": .5})